In [1]:
from PIL import Image
import numpy as np
import cupy as cp
from scipy import misc
from matplotlib import pyplot, cm
from tqdm.notebook import tqdm
from IPython.display import display

In [2]:
def compute_coordinates(resolution, pitch):
    """Return an array of shape (res) containing the coordinates of a sampling lattice with given pitch."""
    indices = np.arange(resolution)
    return (indices - resolution/2 + 0.5) * pitch

def display_image(values, normalize=True, color_map=None):
    """Display an array of values as an image using a color map."""
    values = values * np.nan_to_num(1.0 / np.amax(values)) if normalize else values
    color_values = cm.get_cmap(color_map)(values)
    uint8_values = (color_values * 255).astype(np.uint8)
    display(Image.fromarray(uint8_values).transpose(Image.ROTATE_90))

In [9]:
# Depths
retina_depth = -17.0
front_depth = 10.0
back_depth = 30.0
near_depth = 50.0
far_depth = np.inf


# Eye properties
pupil_diameter = 5.0
retina_size = 5.5
retina_resolution = 768


# Display properties
front_pixel_pitch = 0.2
back_pixel_pitch = 0.2
front_pixel_size = front_pixel_pitch
back_pixel_size = back_pixel_pitch

front_useful_size = retina_size * front_depth / abs(retina_depth) + pupil_diameter
view_useful_size = front_pixel_size * (back_depth / front_depth) + pupil_diameter * (back_depth / front_depth - 1)

front_resolution = 2 * np.floor((front_useful_size + front_pixel_size) * 0.5 / front_pixel_pitch).astype(np.int32) - 1
front_x = compute_coordinates(front_resolution, front_pixel_pitch)
view_x = front_x * (back_depth / front_depth)
view_beginning = np.floor((view_x - (view_useful_size + back_pixel_size) * 0.5) / back_pixel_pitch).astype(np.int32) + 1
view_resolution = 1 - np.amax(view_beginning + np.flip(view_beginning))

print(front_resolution, view_resolution)

41 53


In [ ]:
# Depths
retina_depth = -17.0
front_depth = 10.0
back_depth = 30.0
near_depth = 50.0
far_depth = np.inf


# Eye properties
pupil_diameter = 5.0
retina_size = 5.5
retina_resolution = 768


# Display properties
front_pixel_pitch = 1.0
back_pixel_pitch = 1.0
front_pixel_size = front_pixel_pitch
front_pixel_size = back_pixel_pitch

front_useful_size = retina_size * front_depth / abs(retina_depth) + pupil_diameter
front_resolution = 2 * np.floor((useful_size_a + pixel_size_a) * 0.5 / pixel_pitch_a).astype(np.int32) - 1

front_x = compute_coordinates(front_resolution, front_pitch)



supersampling = 3 # This value must be an odd integer!



useful_size_a = retina_size * z_a / abs(z_r) + pupil_diameter
res_a = 2 * np.floor((useful_size_a + pixel_size_a) * 0.5 / pixel_pitch_a).astype(np.int32) - 1


view_pitch = pixel_pitch * (z_b / z_a)
view_size = pixel_size * (z_b / z_a) + pupil_diameter * (z_b / z_a + 1)
view_res = np.ceil(view_size / pixel_size)
view_res = view_res + 1 if view_pitch % pixel_size != 0 else view_res

res_stride = np.rint(z_b / z_a).astype(np.int32)
size_a = size_r * z_a / abs(z_r) + pupil_diameter
pitch_a = size_a / res_a
stride_b = pitch_a * (z_b / z_a)
pitch_b = stride_b / res_stride
res_b = np.ceil((pupil_diameter / pitch_b) * (z_b / z_a - 1)).astype(np.int32) + res_stride
pitch_r = size_r / res_r

xi_r = 0.5 / pitch_r
xi_a = 0.5 / pitch_a
xi_b = 0.5 / pitch_b

res_lf = (res_a * supersampling, res_b * supersampling, res_a * supersampling, res_b * supersampling)
res_disp = (res_a, res_b, res_a, res_b)

print('Retina resolution:', (res_r, res_r))
print('Display resolution:', res_disp)
print('Light field resolution:', res_lf)

In [ ]:
x = np.random.rand(*res_lf)

print(x.nbytes/2**30)

In [ ]:
retinal_image = misc.face(True).T[256:, ::-1]
display_image(retinal_image)